<a href="https://colab.research.google.com/github/anhvu1214/Xltn/blob/main/DTW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dtw
import librosa
import librosa.display
import numpy as np
import IPython.display
import matplotlib.pyplot as plt
from dtw import dtw
from IPython.display import Image

  Created wheel for dtw: filename=dtw-1.4.0-py3-none-any.whl size=5315 sha256=ac90eb01d312e4daf4ad06cd6d9f7814b7d053c38e7cf6d69744bec3e544aeca
  Stored in directory: /root/.cache/pip/wheels/f9/51/54/b7f446e6d4f5be9a66066b59c5030731191260515737be4496
Successfully built dtw


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
